In [1]:
def parse_edge(s):
    user, follower = s.split("\t")
    return (int(user), int(follower))

def step(item):
    prev_v, prev_d, next_v = item[0], item[1][0], item[1][1]
    prevDist, prevPath = prev_d.split('_', 1)
    nextDist = int(prevDist) + 1
    nextPath = prevPath + ',' + str(next_v)
    return (next_v, str(nextDist) + '_' + nextPath)

def complete(item):
    v, old_d, new_d = item[0], item[1][0], item[1][1]
    oldDist = None
    if old_d != None:
        oldDist = int(old_d.split('_', 1)[0])
    newDist = None
    if new_d != None:
        newDist = int(new_d.split('_', 1)[0])
    return (v, old_d if oldDist is not None else new_d)


In [2]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

In [3]:
n = 10  # number of partitions
edges = sc.textFile("/data/twitter/twitter_sample.txt").map(parse_edge)
forward_edges = edges.map(lambda e: (e[1], e[0])).partitionBy(n).cache()

x = 12
d = 0
tmp = str(d) + '_' + str(x)
distances = sc.parallelize([(x, tmp)]).partitionBy(n)
exit = False

while True:
    candidates = distances.join(forward_edges, n).map(step)
    new_distances = distances.fullOuterJoin(candidates, n).map(complete).distinct().cache()
    
    for i, x in new_distances.collect():
        if i == 34:
            print x.split('_')[1]
            exit = True
            break
            
    if not exit:
        d += 1
        distances = new_distances
        print 'd=%d' % d
    else:
        break

d=1
d=2
d=3
d=4
d=5
d=6
d=7
12,422,53,52,107,20,23,274,34
